# ML Pipeline Preparation

### 1. Import libraries 

In [1]:
# import libraries
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report, accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
from nltk import pos_tag, ne_chunk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

import re
import os

import pickle
import sqlite3

[nltk_data] Downloading package punkt to /home/stephen137/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/stephen137/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/stephen137/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/stephen137/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


### 2. Load data

- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql('SELECT * FROM disaster_response', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,source_language
0,2,Weather update a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,French
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,Italian
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Indonesian
3,9,UN reports Leogane 80 90 destroyed Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,English
4,12,says west side of Haiti rest of the country ...,facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,French


In [3]:
X =  df['message']
X[0]

'Weather update   a cold front from Cuba that could pass over Haiti'

In [4]:
X.shape

(26234,)

In [5]:
type(X)

pandas.core.series.Series

In [6]:
Y = df.drop(['id', 'message', 'original', 'genre', 'source_language'], axis=1)
Y.shape

(26234, 36)

In [7]:
type(Y)

pandas.core.frame.DataFrame

### 2. Write a tokenization function to process your text data

In [8]:
from nltk.tokenize import word_tokenize

def tokenize(text):
    
    """Function that cleans and tokenizes input text 
        
    Args:
        text string
        
        
    Returns: 
        clean tokens
    """
       
    # tokenize text
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    

    # define stop words
    stop_words = stopwords.words("english")
    
    words = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    clean_tokens = []
    
    
    # iterate through each token
    for tok in tokens:
       
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().replace(" ", "")
        clean_tokens.append(clean_tok)         
        
    return clean_tokens

In [9]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
X[115]

'City that i m near is Port au Prince for me  m at Titanyen'

In [11]:
tokenize(X[115])

['city', 'near', 'port', 'au', 'prince', 'titanyen']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('nlp_pipeline', Pipeline([
                ('vect', TfidfVectorizer(tokenizer=tokenize))
            ])),
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
      
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [15]:
pipeline = build_model()
pipeline.fit(X_train, y_train)

/home/stephen137/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('nlp_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  TfidfVectorizer(tokenizer=<function tokenize at 0x7faa8489b8b0>))]))])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [35]:
category_names = list(df.columns[4:40])
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [16]:
y_pred = pipeline.predict(X_test)

In [19]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [44]:
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category {}: {} '.format(i+1, category_names[i]))
    print(classification_report(y_test.iloc[:, i], y_pred[:, i],zero_division=1))
    print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i], y_pred[:, i])))
    print('-'*60)

Category 1: related 
              precision    recall  f1-score   support

           0       0.69      0.45      0.54      1223
           1       0.85      0.93      0.89      3987
           2       0.30      0.43      0.35        37

    accuracy                           0.82      5247
   macro avg       0.61      0.60      0.59      5247
weighted avg       0.81      0.82      0.80      5247

Accuracy 0.8174194777968363


------------------------------------------------------------
Category 2: request 
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4390
           1       0.81      0.49      0.62       857

    accuracy                           0.90      5247
   macro avg       0.86      0.74      0.78      5247
weighted avg       0.89      0.90      0.89      5247

Accuracy 0.898989898989899


------------------------------------------------------------
Category 3: offer 
              precision    recall  f1-score   supp

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__max_depth': [10, 20, 30]
}

cv = GridSearchCV(pipeline, parameters, cv=5, verbose=2, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def evaluate_model(pipeline, X_test, y_test, category_names):

    y_pred = pipeline.predict(X_test)
    
    for i in range(len(category_names)):
        
        print('Category {}: {} '.format(i+1, category_names[i]))
        print(classification_report(y_test.iloc[:, i], y_pred[:, i],zero_division=1))
        print('Accuracy {}\n\n'.format(accuracy_score(y_test.iloc[:, i], y_pred[:, i])))
        print('-'*60)  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
def save_model(model, model_filename):
    
    '''
    Pickle the model for unpickling later

    INPUTS:
        model (pipeline.Pipeline): model to be saved
        model_pickle_filename (str): destination pickle filename
    '''
    pickle.dump(model, open(model_filename, 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

Run the following from the command line to run the machine learning Pipeline: 

    python3 train_classifier.py disaster_response.db random_forest.pkl